# The Zara Case: Single and Multiple Source Analysis

Install gurobipy and import all the functions in gurobipy.

In [34]:
# Installation
%pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# Import all the functions
from gurobipy import *
import numpy as np

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import and prepare the data

In [37]:
# Import the data from an Excel file

# Import the library called Pandas first
import pandas as pd

# Upload the data and save them into "data"
data = pd.read_excel('/content/drive/MyDrive/data.xlsx', index_col=0)

In [38]:
data

,Lille,Paris,Bordeaux,Toulouse,Strasbourg,Lyon,Marseille,Nice,Geneva,Torino,...,Barcelona,Madrid,Bilbao,Valencia,Sevilla,Porto,Lisboa,Capacity,Exten./Construct. Cost,Delivery Cost Budget
Travel Times,,,,,,,,,,,,,,,,,,,,,
Madrid,21.3,18.1,15.1,11.4,23.5,17.7,15.7,18.0,19.7,21.0,...,8.9,0.2,5.7,5.1,6.0,8.1,9.0,123200.0,0.1,4500.0
Lille,0.1,3.2,11.4,12.8,7.5,9.9,14.3,16.5,10.7,14.2,...,17.9,21.2,16.2,22.7,27.1,25.7,27.9,88000.0,0.1,NaN
Orléans,5.0,1.9,6.6,7.9,8.4,6.6,10.8,13.1,7.7,11.1,...,13.1,16.4,11.4,17.9,22.3,20.9,23.1,112000.0,0.5,NaN
Montélimar,11.9,8.7,9.0,5.6,9.1,2.1,2.4,4.6,4.2,5.5,...,7.1,15.6,11.8,11.9,19.6,21.3,23.5,73600.0,0.1,NaN
Châlon,8.1,4.9,8.6,8.6,5.2,1.8,6.3,8.5,2.9,6.3,...,10.9,18.3,13.3,15.8,23.4,22.8,25.0,64000.0,0.2,NaN
Piacenza,16.2,13.2,14.8,12.6,7.8,7.0,7.5,4.6,5.6,2.6,...,14.1,22.6,18.9,18.9,26.6,28.3,30.5,137600.0,0.5,NaN
Bordeaux,15.1,8.4,0.1,3.5,13.8,7.9,9.2,11.5,10.0,12.3,...,9.1,9.8,4.8,11.1,15.7,14.3,16.4,50000.0,4000.0,NaN
Barcelone,17.9,14.8,9.1,5.6,16.1,9.1,7.2,9.5,11.2,12.5,...,0.2,8.9,8.7,5.0,12.7,16.6,17.8,50000.0,15000.0,NaN
Rome,23.1,20.1,21.3,17.8,15.1,13.9,12.8,9.8,12.5,9.5,...,19.3,27.8,24.1,24.2,31.8,33.6,35.7,50000.0,20000.0,NaN


In [39]:
# Select the times, demand, capacity, cost, budget from the excel file

demand=data.iloc[-1,:-3]
time=data.iloc[:-1,:-3]
capacity=data.iloc[:-1,-3]
cost=data.iloc[:-1,-2]
budget=data.iloc[0,-1]


# The number of Distribution Centers (DCs)is equal to the number of rows
num_centers = len(time.index)

# The number of aggregated stores is equal to the number of columns
num_cities = len(time.columns) 


In [40]:
print(num_cities)

21


In [ ]:
print(num_centers)

9


## **Question 1**: Which DCs would you build and/or extend (and how large should be the extensions) with single-source supplying (i.e., the demand of a client should be delivered by a single DC)?

Initializing the model and defining the decision variables

In [41]:
# Initialize the model
model = Model('zara-Q1')

# Create the variable x[i] = 1 if DC i will be extended or constructed
x = model.addVars(num_centers, vtype=GRB.BINARY, name='extension_construct')

# Create the variable y[i] to compute the extension unit if i in range(0,6)
y = model.addVars(6, vtype=GRB.INTEGER, name='extension_unit')

# Create the variable a[i,j] = 1 if city j is delivered by i DCs, 0 otherwise
a = model.addVars(num_centers, num_cities, vtype=GRB.BINARY, name='deliver_site')

Defining the input parameters

In [42]:
# Read in the input parameters

# m[i] is the cost of construction or extension
m = np.array([i for i in cost])
print('cost: ',m)

# t[i,j] is the delivery time from i to j
t = np.array([[j for j in i] for i in time.values])
print('Dcs and Cities: ',t.shape)

# c[i] is the capacity of sites
c = np.array([i for i in capacity])
print('capacities: ',c)

# d[j] is the demand of cities
d = np.array([i for i in demand])
print('demands: ',d)

cost:  [1.0e-01 1.0e-01 5.0e-01 1.0e-01 2.0e-01 5.0e-01 4.0e+03 1.5e+04 2.0e+04]
Dcs and Cities:  (9, 21)
capacities:  [123200.  88000. 112000.  73600.  64000. 137600.  50000.  50000.  50000.]
demands:  [ 20600.  45000.  21700.  22600.  13500.  39200.  30300.  18500.   9900.
  15800.  23600.   6700.  50200.  18300.  86900. 108600.  19900.  20100.
  26100.  32500.  35000.]


Defining all the constraints

In [43]:
# Constraint 1: Zara can construct or extend no more than 3 sites
model.addConstr(quicksum(x[i] for i in range(num_centers)) <= 3) 

# Constraint 2: If Zara decides to extend, then 8000<= yi <= 13000, else yi = 0
for i in range(6):
    model.addConstr(y[i] >= 8000*x[i])
model.addConstrs(y[i] <= 13000*x[i] for i in range(6))

# Constraint 3: Single source supplying
for j in range(num_cities):
    model.addConstr(quicksum(a[i,j] for i in range(num_centers)) == 1)

# Constraint 4: The total monthly delivery costs do not exceed a budget of 4500 euros
model.addConstr(quicksum(d[j]/1000 * quicksum(a[i, j]*t[i, j] for i in range(num_centers)) for j in range(num_cities)) <= 4500)

# Constraint 5: The supplies must exceed the demands
model.addConstrs(quicksum(a[i,j]*d[j] for j in range(num_cities)) <= c[i]+y[i] for i in range(6)) 
model.addConstrs(quicksum(a[i,j]*d[j] for j in range(num_cities)) <= c[i]*x[i] for i in range(6,9))

model.update()

Defining the objective function

In [44]:
# Objective: minimize the total number of inhabitants threatened by the open plants
model.setObjective(quicksum(m[i]*y[i] for i in range(6))+quicksum(m[i]*x[i] for i in range(6,9)), GRB.MINIMIZE)


Calling the solver

In [45]:
# Call the solver 
model.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 44 rows, 204 columns and 609 nonzeros
Model fingerprint: 0xd7b2c661
Variable types: 0 continuous, 204 integer (198 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+05]
Presolve removed 1 rows and 25 columns
Presolve time: 0.00s
Presolved: 43 rows, 179 columns, 513 nonzeros
Variable types: 0 continuous, 179 integer (173 binary)

Root relaxation: objective 5.672295e+03, 76 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5672.29480    0   14          - 5672.29480      -     -    0s
     0     0 5777.55279 

Now we have obtained the solutions for the optimization problem. Displaying the optimal value of the extension/construction cost.

In [46]:
# Save the optimal value
optimal_value_Q1 = pd.DataFrame([model.objVal])
optimal_value_Q1.index.name = "optVal"

# Display the optimal value the extension/construction cost
print("The optimal value for the extension/construction cost is:")
print(model.objVal)

The optimal value for the extension/construction cost is:
19800.0


Display which DCs need to be extended/constructed (binary values)

In [47]:
# Variable: x
# Start creating a dataframe of 0 values with the right size (1 row, as many columns as the number of sites)
optimal_x_Q1 = pd.DataFrame(0, index=range(1), columns=range(num_centers))
optimal_x_Q1.index.name = "x"
# Then, iterate on the columns of the dataframe using a for cycle
for i in range(num_centers):
    # Update the value of the column
    optimal_x_Q1.iloc[0,i] = x[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_x_Q1)

   0  1  2  3  4  5  6  7  8
x                           
0  1  0  0  0  0  0  1  1  0


Display by how much capacity does a DC need to be extended, if it needs to be extended (integer value)

In [48]:
# Variable: y
# Start creating a dataframe of 0 values with the right size (1 row, with 6 columns)
optimal_y_Q1 = pd.DataFrame(0, index=range(1), columns=range(6))
optimal_y_Q1.index.name = "y"
# Then, iterate on the columns of the dataframe using a for cycle
for i in range(6):
    # Update the value of the column
    optimal_y_Q1.iloc[0,i] = y[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_y_Q1)

      0  1  2  3  4  5
y                     
0  8000  0  0  0  0  0


In [49]:
# Report the optimal solution in the excel file

with pd.ExcelWriter('/content/drive/MyDrive/data.xlsx', mode='a', if_sheet_exists='overlay') as writer:
# Upload the dataframes on a new sheet called 'Q1', one under the other
    optimal_value_Q1.to_excel(writer, sheet_name='Q1')
    optimal_x_Q1.to_excel(writer, sheet_name='Q1', startrow=3)
    optimal_y_Q1.to_excel(writer, sheet_name='Q1', startrow=6)

In [73]:
#Visualization for the Monthly delivery costs for all the stores
sum=[0]*21

# Extract the values from the Gurobi variable
a_values = np.array([[a[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum[j] = sum[j]+a_values[i,j]*t[i,j]*d[j]/1000

print("The average Monthly delivery costs for all the stores are:")
print(sum)


The average Monthly delivery costs for all the stores are:
[2.06, 144.0, 143.22, 194.36, 101.25, 70.56, 72.72, 85.1, 41.58, 41.08, 23.6, 23.45, 371.48, 184.83, 1138.39, 21.72, 113.43, 100.5, 331.47, 692.25, 574.0]


In [57]:
#Visualization for the average delivery time for all the stores
sum=[0]*21

# Extract the values from the Gurobi variable
a_values = np.array([[a[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum[j] = sum[j]+a_values[i,j]*t[i,j]

print("The average delivery time for all the stores are:")
print(sum)

The average delivery time for all the stores are:
[0.1, 3.2, 6.6, 8.6, 7.5, 1.8, 2.4, 4.6, 4.2, 2.6, 1.0, 3.5, 7.4, 10.1, 13.1, 0.2, 5.7, 5.0, 12.7, 21.3, 16.4]


In [58]:
#Visualization for the average delivery time from all the DCs
sum=[0]*9

# Extract the values from the Gurobi variable
a_values = np.array([[a[i, j].x for j in range(21)] for i in range(9)])
for i in range(9):
  for j in range (21):
    sum[i] = sum[i]+a_values[i,j]*t[i,j]

print("The average delivery time from all the DCs are:")
print(sum)

The average delivery time from all the DCs are:
[5.9, 10.8, 19.7, 27.9, 10.4, 29.200000000000003, 16.4, 17.7, 0.0]


**Comment**:  In the single source problem, the DC 1 needs to be extended by 8000 units, while DCs 7 and 8 need to be constructed, for optimizing the overall extension/construction costs. The optimization results in the minimum extension/construction cost of 19800 euros.

## **Question 2**: Which DCs would you build and/or extend (and how large should be the extensions) with multi-source supplying (i.e., the quantity delivered to a store can be supplied from several DCs)?


Initializing the model and defining the decision variables

In [59]:
# Initialize the model
model2 = Model('zara-Q2')

# Create the variable x[i] = 1 if DC i will be extended or constructed
x = model2.addVars(num_centers, vtype=GRB.BINARY, name='extension_construct')

# create the variable y[i] to compute the extension unit if i in range(0,6)
y = model2.addVars(6, vtype=GRB.INTEGER, name='extension_unit')

# Create the variable quan[i,j] which denotes the quantity served to city j is from DC i
quant = model2.addVars(num_centers, num_cities, vtype=GRB.INTEGER, name='quantity_deliver_site')

Defining the input parameters

In [60]:
# Read in the constants

# m[i] is the cost of construction or extension
m = np.array([i for i in cost])
print('cost: ',m)

# t[i,j] is the delivery time from i to j
t = np.array([[j for j in i] for i in time.values])
print('Dcs and Cities: ',t.shape)

# c[i] is the capacity of sites
c = np.array([i for i in capacity])
print('capacities: ',c)

# d[j] is the demand of cities
d = np.array([i for i in demand])
print('demands: ',d)

cost:  [1.0e-01 1.0e-01 5.0e-01 1.0e-01 2.0e-01 5.0e-01 4.0e+03 1.5e+04 2.0e+04]
Dcs and Cities:  (9, 21)
capacities:  [123200.  88000. 112000.  73600.  64000. 137600.  50000.  50000.  50000.]
demands:  [ 20600.  45000.  21700.  22600.  13500.  39200.  30300.  18500.   9900.
  15800.  23600.   6700.  50200.  18300.  86900. 108600.  19900.  20100.
  26100.  32500.  35000.]


Defining all the constraints

In [61]:
# Constraint 1: Zara can construct or extend no more than 3 sites
model2.addConstr(quicksum(x[i] for i in range(num_centers)) <= 3) 

# Constraint 2: If Zara decides to extend, then 8000<= yi <= 13000, else yi = 0
for i in range(6):
    model2.addConstr(y[i] >= 8000*x[i])
model2.addConstrs(y[i] <= 13000*x[i] for i in range(6))

# Constraint 3: Multiple source supplying. The summation of the supplies from multiple sources for a city should be greater than its demand
for j in range(num_cities):
    model2.addConstr(quicksum(quant[i,j] for i in range(num_centers)) >= demand[j])

# Constraint 4: The total monthly delivery costs do not exceed a budget of 4500 euros
model2.addConstr(quicksum(quant[i, j] * t[i, j] * 0.001 for i in range(num_centers) for j in range(num_cities)) <= 4500)


# Constraint 5: The supplies must exceed the demands
model2.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]+y[i] for i in range(6)) 
model2.addConstrs(quicksum(quant[i,j] for j in range(num_cities)) <= capacity[i]*x[i] for i in range(6,9))

model2.update()


Set the objective function and call the solver

In [62]:
# Objective: 
model2.setObjective(quicksum(m[i]*y[i] for i in range(6))+quicksum(m[i]*x[i] for i in range(6,9)), GRB.MINIMIZE)

In [63]:
# Call the solver 
model2.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 44 rows, 204 columns and 609 nonzeros
Model fingerprint: 0x7c8702b4
Variable types: 0 continuous, 204 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e-04, 5e+04]
  Objective range  [1e-01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
Presolve time: 0.00s
Presolved: 44 rows, 204 columns, 609 nonzeros
Variable types: 0 continuous, 204 integer (9 binary)

Root relaxation: objective 5.660000e+03, 104 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5660.00000    0    5          - 5660.00000      -     -    0s
H    0     0                    6100.0000000 5660.00000  7.21%    

Now we have obtained the solutions for the optimization problem. Displaying the optimal value of the extension/construction cost.

In [64]:
# Save the optimal value
optimal_value_Q2 = pd.DataFrame([model2.objVal])
optimal_value_Q2.index.name = "optVal"

# Display the optimal value the extension/construction cost
print("The optimal value for the extension/construction cost is:")
print(model2.objVal)

The optimal value for the extension/construction cost is:
5660.1


Display which DCs need to be extended/constructed (binary values)

In [65]:
# Variable: x
# Start creating a dataframe of 0 values with the right size (1 row, as many columns as the number of sites)
optimal_x_Q2 = pd.DataFrame(0, index=range(1), columns=range(num_centers))
optimal_x_Q2.index.name = "x"
# Then, iterate on the columns of the dataframe using a for cycle
for i in range(num_centers):
    # Update the value of the column
    optimal_x_Q2.iloc[0,i] = x[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_x_Q2)

   0  1  2  3  4  5  6  7  8
x                           
0  1  0  0  1  0  0  1  0  0


Display by how much capacity does a DC need to be extended, if it needs to be extended (integer value)

In [66]:
# Variable: y
# start creating a dataframe of 0 values with the right size (1 row, with 6 columns)
optimal_y_Q2 = pd.DataFrame(0, index=range(1), columns=range(6))
optimal_y_Q2.index.name = "y"
# Iterate on the columns of the dataframe using a for cycle
for i in range(6):
    # Update the value of the column
    optimal_y_Q2.iloc[0,i] = y[i].X
    # [var.X is used to take the optimal value of variable var; note that X is capitalized]

# Display the optimal solution
print(optimal_y_Q2)

      0  1  2     3  4  5
y                        
0  8600  0  0  8001  0  0


In [67]:
# Report the optimal solution in the excel file

with pd.ExcelWriter('/content/drive/MyDrive/data.xlsx', mode='a', if_sheet_exists='overlay') as writer:
# Upload the dataframes a new sheet called 'Q2', one under the other
    optimal_value_Q2.to_excel(writer, sheet_name='Q2')
    optimal_x_Q2.to_excel(writer, sheet_name='Q2', startrow=3)
    optimal_y_Q2.to_excel(writer, sheet_name='Q2', startrow=6)

In [68]:
#Visualization for the average delivery time for all the stores
sum=[0]*21
sum_quant_across_rows = [0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum_quant_across_rows[j]=sum_quant_across_rows[j]+quant_values[i,j]
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]
  sum[j]=sum[j]/sum_quant_across_rows[j]

print("The average delivery time for all the stores are:")
print(sum)

The average delivery time for all the stores are:
[0.1, 2.8302222222222224, 11.4, 7.9, 7.5, 1.8, 5.075247524752475, 4.6, 2.9, 2.6, 1.0, 3.5, 7.4, 10.1, 7.935420023014959, 5.685491712707181, 11.4, 11.9, 6.0, 18.958043076923072, 9.0]


In [69]:
#Visualization for the average delivery time from all the DCs
sum=[0]*9
sum_quant_across_columns = [0]*9

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for i in range(9):
  for j in range (21):
    sum_quant_across_columns[i]=sum_quant_across_columns[i]+quant_values[i,j]
    sum[i] = sum[i]+quant_values[i,j]*t[i,j]
  sum[i]=sum[i]/sum_quant_across_columns[i]

print("The average delivery time from all the DCs are:")
print(sum)

The average delivery time from all the DCs are:
[3.8449908952959033, 5.1560227272727275, 13.161348214285715, 7.654527518045121, 3.0178125, 5.547165697674418, 9.687811756235124, nan, nan]


<ipython-input-69-35d5b4e29502>:11: RuntimeWarning: invalid value encountered in double_scalars
  sum[i]=sum[i]/sum_quant_across_columns[i]


In [74]:
#Visualization for the Monthly delivery costs for all the stores

sum=[0]*21

# Extract the values from the Gurobi variable
quant_values = np.array([[quant[i, j].x for j in range(21)] for i in range(9)])
for j in range(21):
  for i in range (9):
    sum[j] = sum[j]+quant_values[i,j]*t[i,j]*0.001

print("The average Monthly delivery costs for all the stores are (Multiple source):")
print(sum)


The average Monthly delivery costs for all the stores are (Multiple source):
[2.06, 127.36000000000001, 247.38, 178.54, 101.25, 70.56, 153.78, 85.10000000000001, 28.71, 41.08, 23.6, 23.45, 371.48, 184.83, 689.588, 617.4444, 226.86, 239.19, 156.6, 616.1363999999999, 315.0]


**Comment**:  In the multiple source problem, the DCs 1 and 4 need to be extended by 8600 units and 8001 units respectively, while DC 7 needs to be constructed, for optimizing the overall extension/construction costs. The optimization results in the minimum extension/construction cost of 5660.1 euros.